## 1. Birthday Attack
A birthday attack is a **bruteforce collision** attack that exploits the mathematics behind the birthday problem in probability theory. A birthday attack involves finding **two different inputs** that produce the **same hash** output in a hashing algorithm. This attack leverages the principle of the **birthday paradox** to find these collisions faster than would be expected by random chance.

## 2. How Does a Birthday Attack Work?
1. **Hash Functions**: In cryptography, hash functions are used to create a fixed-size string (a hash) from variable-size input. Ideally, each unique input should produce a unique hash.

2. **Finding Collisions**: The attack seeks to find two different inputs that result in the **same hash** output. The probability of finding such a collision is much higher than intuitively expected, similar to the birthday paradox.

3. **Probability Advantage**: For a hash function that produces a hash of n bits, a birthday attack can find a collision in approximately $2^{n/2}$ operations rather than $2^n$ operations, as one might expect with random guessing. 

## 3. Implications in Cryptography
1. **Digital Signatures**: Birthday attacks can potentially compromise digital signatures by finding two messages with the same hash, one benign and one malicious.

2. **Data Integrity**: Any cryptographic system relying on hash functions for data integrity might be vulnerable to birthday attacks.

3. **Certificate Authority**: The attack could undermine the trust in certificate authorities if generating two certificates with the same hash is possible.


## 4. Implementation
### 4.1 Bruteforce
The most common way to implement a birthday attack is by utilizing a bruteforce approach (i.e. generate input texts on the fly until a hash collision is found).

In [1]:
import hashlib
import random

Let's first create a `32-bit` hash function. It is faster to be hacked. The same principles apply for 64-bit and 128-bit hash functions, only the time for cracking them is from several minutes to several hours.

In [2]:
def hash_32bit(text: str) -> str:
    return hashlib.md5(text.encode('utf8')).hexdigest()[:8] # if we want 64-bit hash, we can use [:16]

Now, let's write a utility function that generates random strings.

In [3]:
CHARSET = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
def generate_random_string(string_length: int) -> str:
    return ''.join(random.choice(CHARSET) for _ in range(string_length))

In [4]:
def birthday_attack_brute_force(num_attempts: int):
    # Initialize a set to store digest values
    hashmap = {}

    # Generate a sample of random text inputs. The sample size is num_attempts.
    texts = [generate_random_string(15) for _ in range(num_attempts)]

    # Calculate the digest for each input
    counter = 0
    for text in texts:
        counter += 1
        hash_key = hash_32bit(text)
        # CASE 1: Check if hashmap already contains the hash key
        if hash_key in hashmap:
            print(f"Collision found after {counter} attempts. Both `{text}` and `{hashmap[hash_key]}` have the same hash `{hash_key}`")
            return
        # CASE 2: Otherwise, add the hash key and text to the hashmap
        hashmap[hash_key] = text

    print("No collisions found")

birthday_attack_brute_force(1000000)

Collision found after 66797 attempts. Both `dI8m87ob1fcaoHC` and `8M6s4UzckquKVUT` have the same hash `fd99e141`


### 4.2 Rainbow table
A Rainbow Table Attack is a cryptographic attack that uses a precomputed table of hash values to crack passwords. Unlike brute force attacks that generate all possible passwords and compute their hashes on the fly, rainbow tables store a list of precomputed hashes for every possible password. This method significantly **reduces the time** needed to crack a password hash.

### 4.3 Distinguished point attack
A distinguished point attack is a technique used to enhance the efficiency of a birthday attack by focusing on specific, often "distinguished" points in the hash function's output space.  It's a way to target the hash space for a faster collision search. 

**Distinguished Points**: These are specific values within the hash function's output space that are considered "interesting" or "distinguished" for the attack. They might be based on certain patterns, constraints, or properties that make them more likely to be part of a collision (e.g. a particular number of leading zeros).

### 4.4 Pollard's Rho algorithm
What if start hashing the output of a has for multiple times and we stubmle upon a cycle: `hash(hash(hash(hash(x)))) = hash(x)`. Well, there is a common [Floyd cycle-founding algorithm](https://en.wikipedia.org/wiki/Cycle_detection#Floyd's_tortoise_and_hare) that uses two pointers to move through the sequence at different speeds. It is also called the *"tortoise and the hare algorithm"*. 

**Pollard's Rho algorithm**, used for **integer factorization**, exploits the "birthday paradox" to find factors faster than trial division. It generates a sequence of numbers using a pseudorandom function and looks for a cycle in that sequence, hoping to find a non-trivial common divisor with the number being factored. 

A python implementation could be seen at [github](https://github.com/SeaweedbrainCY/RhoPollard).



<center><img src="img/pollard_rho.png" alt=" Pollard's Rho algorithm" width="1000" height="685" /></center>

## References
* [(wiki) Birthday attack](https://en.m.wikipedia.org/wiki/Birthday_attack)
* [[Security Brief] - P13 - Birthday Attack](https://lepuchin.com/Security-Brief-P13-Birthday-Attack)
* [Birthday attack in Cryptography](https://www.geeksforgeeks.org/birthday-attack-in-cryptography/)
* [Reasons Why Rainbow Table Attacks Are Dangerous ](https://medium.tuanh.net/reasons-why-rainbow-table-attacks-are-dangerous-and-how-salting-passwords-protects-against-them-548db34d7ac4)
* [Pollard's Rho Algorithm for Prime Factorization](https://www.geeksforgeeks.org/pollards-rho-algorithm-prime-factorization/)
* [(reddit) Understanding Birthday Attach](https://www.reddit.com/r/crypto/comments/bbau44/comment/ekjykwk/)